---
# URL Reputation
### Adrien Manciet - Thibault Sourdeval
---

### Introduction

Ce dataset est un ensemble d'url qui sont labellisés. Si le label vaut 1, l'url est dangereux, si il vaut -1, il ne l'est pas. L'objectif sera de faire un algorithme de classification des url en apprenant sur le dataset disponible. 

---
# Partie 1 - Phase d'exploration
---

Nous notons que les fichiers de données sont sous la forme de matrices sparse. Cela signifie que seules les valeurs non nulles sont gardées en mémoire. 
Cela permet d'épargner des erreurs de mémoire. 

Le fichier features contient des numéros qui semblent correspondre à des subdivisions de l'url contenant en blocs. Exemple : la première ligne du fichier features affiche 4, ce qui pourrait correspondre aux quatres premiers caractères de l'url 'http'. 

Nous codons une fonction de prévisualisation pour mieux comprendre la structure des données en les transformant en un dataframe. Pour la suite, 
nous resterons dans le format de données initial.

**Chargement des modules**

In [ ]:
from sklearn.datasets import load_svmlight_file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


**Fonction de prévisualisation** 

In [ ]:
def preview_data(day, nb_lines, nb_cols, random = True):
    # Si random est laissé tel quel, une valeur au hasard 
    # est prise pour la première ligne et la première colonne à afficher

    path =f"url_svmlight/url_svmlight/Day{day}.svm"
    X, y = load_svmlight_file(path)
    print("Les données sont de taille : ", X.shape)

    if random == True:
        start_line = np.random.randint(0, len(y)-nb_lines)
        start_col = np.random.randint(0,X.shape[1])
    else : 
        start_line = int(input("Première ligne à afficher : "))
        start_col = int(input("Première colonne à afficher : "))
    
    label_list = []
    for i in range(start_col, start_col+nb_cols):
        label_list.append(i)

    X_df = pd.DataFrame(X[start_line: start_line+nb_lines, start_col: start_col+nb_cols].toarray(), columns=label_list)
    y_df = pd.DataFrame(y[start_line: start_line+nb_lines], columns=['label'])

    data = pd.concat([X_df, y_df], axis=1)
    return data

preview_data(17, 10, 10, random=True)


**Visualisations grahiques** 

In [ ]:
def scatter_plot(day, feature_x, feature_y): 
    path =f"url_svmlight/url_svmlight/Day{day}.svm"
    X, y = load_svmlight_file(path)

    X = X[0:2000,:2000].toarray()
    plt.figure()
    plt.grid(alpha=0.2)
    sc = plt.scatter(X[:,feature_x], X[:,feature_y], c=y[0:2000], cmap="viridis",alpha=0.5)
    plt.xlabel(f'Feature {feature_x}')
    plt.ylabel(f'Feature {feature_y}')
    

    cbar = plt.colorbar(sc)
    cbar.set_label("Label")
    plt.show()

scatter_plot(11,4,3)

**Commentaires sur les features**

Elles ne correspondent pas à des critères interprétables par l'humain. Probablement, ce sont des indicateurs de présence de certains mots, ou certains caractères dans un découpage de l'url qui est défini dans le fichier `features_types`.

---
# Partie 2 - Feature Engineering
---

Comme beaucoup de nos colonnes de features ne contiennent que des 0 ou des 1, et que cela ne permettra pas la classification, nous décidons d'enlever ces colonnes dans les données qui serveront à l'apprentissage. 

Pour ce faire, nous utilisons un critère sur la variance minimale d'une colonne dans chaque fichier. Puis, nous regardons le nombre de fichier pour lesquels une colonne a été gardée. Nous mesurons cela en pourcentage. 
>Exemple : la colonne 1 a une variance supérieure au critère minimal dans les fichiers 1 à 10, mais pas dans les fichiers 11 à 20. Ainsi, l'algorithme a gardé la colonne un pour les fichiers 1 à 10 et l'a enlevée dans les autres. Au total, la colonne 1 a été gardée dans 50% des cas.

Dans un premier temps, nous gardons les colonnes dès lors qu'elles sont gardées au moins une fois, soit que leur pourcentage d'apparition est strictement positif. Nous pourrons raffiner cela pour garder moins de features si on voit que cela améliore la performance de la méthode d'apprentissage.

### Sélection des features 

**Trouver les features à garder**

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import load_svmlight_file
import os

data_dir = "url_svmlight/url_svmlight"
max_features = 3300000  # à adapter à ton dataset

selector = VarianceThreshold(threshold=0.01)

# Préparer la liste des fichiers à traiter
files_to_process = sorted(os.listdir(data_dir))[:50]  # exemple sur 10 jours
num_files = len(files_to_process)

# Initialiser UN SEUL tableau pour compter les sélections
# C'est beaucoup plus efficace en mémoire
column_keep_counts = np.zeros(max_features, dtype=int)

print(f"Traitement de {num_files} fichiers...")

# Boucle sur les fichiers
for file in files_to_process:
    X, y = load_svmlight_file(os.path.join(data_dir, file), n_features=max_features)
    
    # On a seulement besoin de "fit", pas de "fit_transform" si on n'utilise pas X_reduced
    selector.fit(X) 
    
    keep_mask = selector.get_support()  # booléen : True si la colonne est gardée
    
    # --- La voici, l'optimisation ---
    # On ajoute le masque (True=1, False=0) à notre compteur total
    column_keep_counts += keep_mask
    # ---------------------------------
    print(f"Traitement {file} terminé.")

print("Traitement terminé.")

# Calculer le pourcentage (exactement comme avant, mais sans la grosse matrice)
column_keep_percentage = (column_keep_counts / num_files) * 100

# Créer DataFrame en ne gardant que les colonnes qui ont été sélectionnées au moins une fois
df_keep = pd.DataFrame({
    'column_index': np.arange(len(column_keep_percentage)),
    'percent_kept': column_keep_percentage
})

# Filtrer les colonnes jamais gardées
df_keep = df_keep[df_keep['percent_kept'] > 0]	

**Filtrer et concaténer toutes les données chargées en ne conservant que les features sélectionnées précédemment**

In [ ]:
from scipy.sparse import vstack, csr_matrix
from sklearn.datasets import load_svmlight_file

data_dir = "url_svmlight/url_svmlight"

X_filtered_all = None
y_all = []

# -------------------------------
# Boucle sur les fichiers
# -------------------------------
for file in sorted(os.listdir(data_dir))[:30]:
    print(f"Traitement de {file}...")
    X, y = load_svmlight_file(os.path.join(data_dir, file), n_features=max_features)
    
    # Filtrage sur les mêmes colonnes
    X_filtered = X[:, df_keep['column_index']]
    
    # Concaténation verticale
    if X_filtered_all is None:
        X_filtered_all = X_filtered
    else:
        X_filtered_all = vstack([X_filtered_all, X_filtered])
    
    y_all.append(y)

# -------------------------------
# Concaténer les labels
# -------------------------------
y_all = np.concatenate(y_all)

# -------------------------------
# Vérification finale
# -------------------------------
print("Shape finale :", X_filtered_all.shape)
print("Nombre total d'échantillons :", X_filtered_all.shape[0])
print("Nombre de colonnes (features) :", X_filtered_all.shape[1])


**Séparation du jeu de données en train et test**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

# Définir la taille du jeu de test (ex: 20%)
TAILLE_TEST = 0.2

# 1. Créer le split principal (Train / Test)
# Le jeu de test (X_test, y_test) ne sera plus touché avant l'évaluation finale.
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_filtered_all, 
    y_all, 
    test_size=TAILLE_TEST, 
    stratify=y_all,  # Garde les proportions de classes
    random_state=42 # Pour la reproductibilité
)


Nous appliquons un scaler, le `MaxAbsScaler` est plus adapté que le `StandardScaler` pour des données clairsemées (sparse), c'est-à-dire contenant beaucoup de zéros. Ceci est bien le cas de notre jeu de données, d'où l'utilisation de cette méthode de pré-traitement des données.

In [ ]:

scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

print(f"Taille dataset complet : {X_filtered_all.shape[0]} échantillons")
print(f"Taille jeu d'entraînement complet : {X_train_full.shape[0]} échantillons")
print(f"Taille jeu de test : {X_test.shape[0]} échantillons")

**Option : réduction de la taille des jeux de données précédents pour entraîner des modèles de manière exploratoire**

In [ ]:
# Définir la fraction du jeu d'entraînement que vous voulez garder
# Ex: 0.1 (soit 10% du X_train_full, ou 8% du dataset total)
TAILLE_TRAIN_REDUIT = 0.1 

# 2. Créer l'échantillon réduit à partir du jeu d'entraînement
# On utilise train_size cette fois-ci.
# Les '_' sont pour les variables que nous n'utiliserons pas (le reste des données)
X_train_reduced, _, y_train_reduced, _ = train_test_split(
    X_train_full, 
    y_train_full, 
    train_size=TAILLE_TRAIN_REDUIT, 
    stratify=y_train_full, # Garde les proportions de classes
    random_state=42
)

print(f"\nTaille jeu d'entraînement réduit : {X_train_reduced.shape[0]} échantillons")

---
# Partie 3 - Phase d'apprentissage
---

## Passage en revue des méthodes d'apprentissage 

Nous commençons par faire un passage en revue des différentes méthodes sur notre jeu de données. Une de nos contraintes principales concerne la taille du jeu de données. Ainsi, nous commençons par évaluer le temps d'execution ainsi que les performances (précisions, variances) des différentes méthodes sur un jeu de données réduit. Nous appliquons ensuite la meilleure méthode au jeu de données entier.

### SVM

**SVM Linéaire**

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

start = time.time()

svm = LinearSVC(C=0.1, random_state=42)
svm.fit(X_train_reduced, y_train_reduced)
y_pred = svm.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

Le résultat est déjà satisfaisant, ce qui peut laisser présupposer d'une structure multi-linéaire de notre jeu de données.

In [ ]:
from sklearn.svm import SVC

start = time.time()

svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train_reduced, y_train_reduced)
y_pred = svm.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

Ici, le temps d'execution est plus long, sans d'amélioration significative de la précision et de la variance. C'est un résultat auquel on pouvait s'attendre puisque svm linéaire donnait de bons résultats, indiquant que notre jeu de données pouvait être séparé linéairement.

### Naive Bayes

**Avec une distribution de probabilité de Bernoulli**

Nos données contiennent beaucoup de 0 et de 1, ce qui se prête bien à une modélisation par une loi de Bernoulli (pile ou face avec une certaine probabilité). Avec une loi Gaussienne, on peut avoir des valeurs entre 0 et 1 avec une plus grande probabilité, ce qui correspond moins à notre jeu de données.

In [ ]:
from sklearn.naive_bayes import BernoulliNB

start = time.time()

model = BernoulliNB(alpha=0.1, binarize=None)  # binarize=None car tes features sont déjà binaires
model.fit(X_train_reduced, y_train_reduced)

y_pred = model.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

**Processus Gaussiens**

Même pas essayé parce que trop long par les exemples de ton fichiers, en plus pas précis.

### Algorithmes basés sur les arbres de décisions

**Arbre de décision simple**

In [ ]:
from sklearn import tree

start = time.time()

model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)
model.fit(X_train_reduced, y_train_reduced)
y_pred = model.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

**AdaBoost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree

start = time.time()

model = AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth=3), n_estimators=100)
model.fit(X_train_reduced, y_train_reduced)
y_pred = model.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

**XGBoost**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree

start = time.time()

model = GradientBoostingClassifier(n_estimators=100)
model.fit(X_train_reduced, y_train_reduced)
y_pred = model.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

**RandomForest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier


start = time.time()

model = RandomForestClassifier(n_estimators=100, criterion='entropy')
model.fit(X_train_reduced, y_train_reduced)
y_pred = model.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

Plus long que les autres méthodes ensemblistes. 

### Conclusion

Nous avons obtenus de bons résultats en des temps raisonnables avec : 
- SVM linéaire : bonne précision, symétrie dans les erreurs, très rapide
- Arbre de décision simple : bonne précision, symétrie acceptable, très rapide, mais nous savons qu'un arbre seul a une faible capacité de généralisation et une tendance à surapprendre contrairement aux SVM
- XGBoost : bonne précision, moins symétrique, moins rapide
- RandomForest : bonne précision, symétrie dans les erreurs, le moins rapide des trois

SVM linéaire, bien qu'étant une méthode simple, semble être la plus efficace sur notre jeu de données. 

## Approfondissement du SVM Linéaire 

Nous allons maintenant refaire tourner SVM linéaire sur un plus gros jeu de données, dans l'objectif de valider notre choix en faisant des validations croisées pour plus de robustesse concernant la performance de cet algorithme pour classifier les urls.

Au lieu de le faire tourner sur `X_train_reduced`, nous allons essayer de le faire tourner avec `X_train_full`, qui ne contient pour l'instant que 10 fichiers d'url.

In [ ]:

start = time.time()

svm = LinearSVC(random_state=42, C=0.1)
svm.fit(X_train_full, y_train_full)
y_pred = svm.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

Nous pouvons constater une légère amélioration des performances de l'algorithme ayant appris sur un jeu de données plus grand. 
Les premiers tuning de paramètres nous indiquent que : 
- une pénalité L1 ne converge que très lentement sans améliorer les performances, nous restons donc sur la pénalité L2 - ceci peut se comprendre par le fait qu'une pénalité L1 consiste à sélectionner les features qui permettent le plus de prédictivité de la classe, mais en mettant leur coefficient à 0, au lieu de très proche de 0 pour L2
- le paramètre C qui permet d'ajuster le compromis entre la largeur de la marge qui sépare les classes des données, et les erreurs de classification : si on veut une marge très large, on doit accepter que certaines valeurs soient classées du mauvais côté de l'hyperplan. La précision s'améliore quelque peu en ajustant C, mais c'est surtout la vitesse de convergence qui est meilleure dans certains cas. Attention cependant à ne pas overfitter.

Le finetuning de C sera l'objet de la partie 4

Néanmoins, nous pouvons d'ores et déjà faire une validation croisée sur notre jeu de données d'entraînement pour apprécier les capacités de généralisations de notre algorithme. 

**Validation Croisée**

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

start = time.time()

# Initialisation du modèle
svm = LinearSVC(random_state=42, C=1)

# Définir une validation croisée stratifiée (préserve la proportion des classes)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Lancer la validation croisée
scores = cross_val_score(svm, X_train_full, y_train_full, cv=cv, scoring='accuracy', n_jobs=-1)

end = time.time()
print("\n--- Validation croisée ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print(f"Scores individuels : {scores}")
print(f"Moyenne de précision : {np.mean(scores):.4f} ± {np.std(scores):.4f}")


Nous constatons une bonne précision homogène sur chacun des folds, ce qui nous permet de conforter le choix du svm linéaire. Il est précis mais aussi robuste car très peu variable selon les jeux de données d'entraînement sélectionnés.

Nous affichons maintenant la matrice de confusion qui permet de comprendre la répartitions des erreurs entre faux positifs et faux négatifs. Nous cherchons un algorithme qui fait des erreurs de façon symétrique, même si on peut débattre de cela.

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score


start = time.time()

svm = LinearSVC(random_state=42, C=0.1)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Une seule passe de cross-validation
y_pred_cv = cross_val_predict(svm, X_train_full, y_train_full, cv=cv, n_jobs=-1)

# Calcul de la précision moyenne sur toutes les prédictions CV
acc = accuracy_score(y_train_full, y_pred_cv)

# Matrice de confusion
cm = confusion_matrix(y_train_full, y_pred_cv)
ConfusionMatrixDisplay(cm).plot()

end = time.time()

print("\n--- Validation croisée ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print(f"Précision moyenne (CV) : {acc:.4f}")


---
# Partie 4 - Tuning d'un hyperparamètre
---

### L'hyperparamètre $C$

**Description**

Nous souhaitons trouver l'hyperparamètre $C$ optimal pour le svm linéaire sur notre jeu de données. 

Rappelons ce que contrôle $C$ dans l'algorithme. $C$ est un terme de pénalisation des erreurs de classifications : 
- si $C$ est petit, des erreurs de classifications sont tolérées. La marge du svm est plus large, ce qui en fait un algorithme avec une capacité de généralisation plus importante. 
- si $C$ est grand, moins d'erreurs de classifications sont tolérées car elles sont pénalisées plus sévèrement dans l'algorithme d'optimisation constitutif du svm. Donc, la marge est plus réduite. La précision est normalement plus grande, mais il y a un risque de surapprentissage plus élevé puisqu'on force l'algorithme à potentiellement fixer les hyperplans séparateurs sur des données qui pourraient être du bruit. 

**Tuning sur dataset restreint**

Pour éviter les problèmes de mémoires, on entraîne sur le dataset réduit.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Modèle de base
svm = LinearSVC(random_state=42, max_iter=5000)

# Grille de C à tester
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Validation croisée stratifiée
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# GridSearch
grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_reduced, y_train_reduced)

print("Best C:", grid_search.best_params_['C'])
print("Best CV score:", grid_search.best_score_)


Comme les tests de la partie précédente le laisser présager, $C=0.1$ est l'otpimum grossier de $C$. Essayons d'affiner cette recherche.

In [ ]:
# Modèle de base
svm = LinearSVC(random_state=42, max_iter=5000)

# Grille de C à tester
param_grid = {'C': [0.23, 0.24, 0.25, 0.26, 0.27, 0.28]}

# Validation croisée stratifiée
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# GridSearch
grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_reduced, y_train_reduced)

print("Best C:", grid_search.best_params_['C'])
print("Best CV score:", grid_search.best_score_)

Une recherche plus fine autour de $0,1$ a montré que $0,2$ est plus performant. Ainsi, nous avons affiné la recherche autour de $0,2$ pour trouver que $C=0,27$ nous donne les meilleures performances sur le jeu de données réduit. 

Il reste à confirmer cela en essayant sur le jeu de données complet.

**Application au dataset complet**

In [ ]:
start = time.time()

svm = LinearSVC(random_state=42, C=0.27)
svm.fit(X_train_full, y_train_full)
y_pred = svm.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))


On constate que ce changement de paramètre a amélioré la précision de l'ordre de $0,01\%$.

In [ ]:
start = time.time()

# Initialisation du modèle
svm = LinearSVC(random_state=42, C=0.27)

# Définir une validation croisée stratifiée (préserve la proportion des classes)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Lancer la validation croisée
scores = cross_val_score(svm, X_train_full, y_train_full, cv=cv, scoring='accuracy', n_jobs=-1)

end = time.time()
print("\n--- Validation croisée ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print(f"Scores individuels : {scores}")
print(f"Moyenne de précision : {np.mean(scores):.4f} ± {np.std(scores):.4f}")

On constate que la précision moyenne sur les folds a diminué de l'ordre de $0,01\%$. Ceci est cohérent avec ce qu'attendu, nous avons augmenté la précision sur notre jeu de données d'entraînement, mais cela nous a coûté un peu de capacité de généralisation. Un compromis doit être fait entre biais et variance et $C=0,27$ semble être un bon choix de ce point de vu.

### Les autres hyperparamètres

**Fonction de perte `loss`**

Elle définit l'erreur du modèle. Par défaut svm linéaire utilise `squared_hinge`. On pourrait utiliser `hinge` simple pour voir si cela améliore les performances du svm linéaire. Une loss en hinge est censée être plus robuste face aux outliers.

**Paramètre `dual`**

Il vaut `True` par défaut, mais en le mettant à `False`, les performances pourraient être meilleures, car c'est particulièrement adapté pour des jeux de données un nombre de samples supérieur aux features, ce qui est notre cas.

**Type de régularisation**

Il est possible de choisir une régularisation l1 ou l2. Nous resterons sur l2 car cela offre une plus grande stabilité.

**Test sur la combinaison de ces nouveaux paramètres**

Nous gardons le $C$ trouvé précédemment, mais nous ajoutons les nouveaux paramètres cités plus haut. 

In [ ]:
start = time.time()

svm = LinearSVC(random_state=42, C=0.27, loss='hinge', penalty='l2')
svm.fit(X_train_full, y_train_full)
y_pred = svm.predict(X_test)

end = time.time()

print("\n--- Évaluation du modèle ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

In [ ]:
start = time.time()

# Initialisation du modèle
svm = LinearSVC(random_state=42, C=0.27, loss='hinge', penalty='l2')

# Définir une validation croisée stratifiée (préserve la proportion des classes)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Lancer la validation croisée
scores = cross_val_score(svm, X_train_full, y_train_full, cv=cv, scoring='accuracy', n_jobs=-1)

end = time.time()
print("\n--- Validation croisée ---")
print(f"Temps d'exécution : {end - start:.2f} s")
print(f"Scores individuels : {scores}")
print(f"Moyenne de précision : {np.mean(scores):.4f} ± {np.std(scores):.4f}")

On constate une amélioration de la précision et de la moyenne de la précision sur les folds. Ces paramètres ont donc effectivement un peu amlioré la performance de notre algorithme. 

Néanmoins, nous avons un warning de convergence, je sais pas trop si c'est une bonne chose, je pense que non.

---
# Partie 5 : Conclusions
---

Pour conclure, dans ce notebook nous devions entrainer un modèle pour classifier les url entre sain et malsain. 

C'est une tâche de machine learning supervisée puisque les données sont labelisées (-1 pour malsain/+1 pour sain).

Nous avons commencé par mettre en place un feature engineering pour extraire les features utiles à l'entrainement du modèle de machine learning. 

Puis nous avons entrainé et testé plusieurs algorithmes de ML (svm, arbres, naive bayes, Gaussian processes).

Grâce à nos résultats, nous avons montré qu'un simple SVM linéaire suffit à obtenir une bonne précision (0.98). 

Ensuite nous avons fine tunné les hyperparamètres du SVM et notamment C. Nous avons pu améliorer notre précision de 0.001%.

Enfin, pour améliorer notre travail si nous avions accès aux personnes "métier", nous aurions pu leur demandé comment ils ont fait pour passer d'un texte url à un vecteur représentant nos features dans la matrice svm. En effet, si nous comprenons mieux à quelles parties de l'url font référence les feature, nous pourrions mieux les filtrer lors de notre feature engineering.
